<a href="https://www.kaggle.com/code/oswind/stockchat-a-stock-market-assistant?scriptVersionId=244477129" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Environment Setup

In [1]:
# Setup the notebook based on running environment.
import os
try:
    from kaggle_secrets import UserSecretsClient
except Exception as e:
    class UserSecretsClient:
        @classmethod
        def get_secret(cls, id: str):
            try:
                return os.environ[id]
            except KeyError as e:
                print(f"KeyError: authentication token for {id} is undefined")
    # Local Run: update the venv.
    %pip install -qU google-genai==1.7.0 chromadb==0.6.3
    %pip install -qU langchain-community langchain-text-splitters wikipedia pandas google-api-core
else:
    # Kaggle Run: update the system.
    !pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
    !pip install -qU google-genai==1.7.0 chromadb==0.6.3
    !pip install -qU langchain-community langchain-text-splitters wikipedia

import ast, chromadb, csv, json, pandas, pytz, re, requests, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.tz import gettz
from enum import Enum
from google import genai
from google.api_core import retry, exceptions
from google.genai import types
from IPython.display import HTML, Markdown, display
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from pydantic import BaseModel, field_validator
from queue import Queue
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable
from wikipedia.exceptions import DisambiguationError, PageError

In [2]:
# Prepare the gemini api for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503, 500})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Import the required google api key.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# A Gemini python api-helper with retry support.
class Gemini:
    gen_model = [["gemini-2.0-flash",15,2000,10000,30000,0,0],     # latest: 15 RPM/1500 RPD/500 search per day/1M TPM
                 ["gemini-2.0-flash-exp",10,10,10,10,0,0],         #    exp: 10 RPM/...
                 ["gemini-2.0-flash-001",15,2000,10000,30000,0,0], # stable: 15 RPM/... (quota shared with latest)
                 ["gemini-2.5-flash-preview-04-17",10,1000,2000,10000,0,0], # 10 RPM/500 RPD/500 search per day/250K TPM
                 ["gemini-2.5-pro-exp-03-25",5,5,5,5,0,0]] #  5 RPM/25 RPD/500 search per day/250K TPM/1M TPD
    gen_local = []
    embed_model = ["text-embedding-004",1500] # 1500 RPM / Max 100 per batch embed request
    error_total = 0
    min_rpm = 3
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 30.0
    dt_rpm = 60.0

    class Limit(Enum):
        FREE = 1
        TIER_1 = 2
        TIER_2 = 3
        TIER_3 = 4
    
    class Model(Enum):
        GEN = 1
        EMB = 2
        LOC = 3

    class Const(Enum):
        STOP = "I don't know."
        METRIC_BATCH = 20
        SERIES_BATCH = 40
        EMBED_BATCH = 100
        CHUNK_MAX = 1500

        @classmethod
        def Stop(cls):
            return cls.STOP.value

        @classmethod
        def MetricBatch(cls):
            return cls.METRIC_BATCH.value

        @classmethod
        def SeriesBatch(cls):
            return cls.SERIES_BATCH.value

        @classmethod
        def EmbedBatch(cls):
            return cls.EMBED_BATCH.value

        @classmethod
        def ChunkMax(cls):
            return cls.CHUNK_MAX.value

    def __init__(self, with_limit: Limit, default_model: int = 0):
        self.client = genai.Client(api_key=GOOGLE_API_KEY)
        self.limit = with_limit.value
        self.m_id = default_model
        self.default_model = default_model
        self.default_local = default_model
        self.gen_rpm = self.gen_model[self.m_id][self.limit]

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + self.gen_model[self.m_id][0]
        elif model == self.Model.LOC:
            return self.gen_local[self.default_local]
        else:
            return "models/" + self.embed_model[0]

    def set_default_model(self, model_index: int):
        if model_index in range(0, len(self.gen_model)):
            self.stop_running()
            self.default_model = model_index
            self.m_id = model_index
        else:
            print(f"set default model({model_index}) must be 0..{len(self.gen_model)-1}")

    def set_default_local(self, model_index: int):
        if model_index in range(0, len(self.gen_local)):
            self.default_local = model_index
        else:
            print(f"set default local({model_index}) must be 0..{len(self.gen_local)-1}")

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        for attempt in range(len(self.gen_model)):
            try:
                if self.gen_rpm > self.min_rpm:
                    self.gen_rpm -= 1
                else:
                    self.on_error(kwargs)
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except exceptions.RetryError as retry_error:
                retriable = retry_error.code in {429, 503, 500}
                if not retriable or attempt == len(self.gen_model)-1:
                    raise retry_error
                self.on_error(kwargs)
            except Exception as e:
                raise e

    def on_error(self, kwargs):
        self.stop_running()
        self.save_error()
        self.next_model()
        print("api.on_error.next_model: model is now ", self.gen_model[self.m_id][0])
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True
        kwargs["model"] = self(Gemini.Model.GEN)
        time.sleep(self.dt_between)

    def stop_running(self):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False

    def validation_fail(self):
        gen_model = self.gen_model[self.m_id]
        gen_model[len(gen_model)-2] += 1
        self.error_total += 1

    def save_error(self):
        gen_model = self.gen_model[self.m_id]
        gen_model[len(gen_model)-1] += 1
        self.error_total += 1

    def next_model(self):
        self.m_id = (self.m_id+1)%len(self.gen_model)
        self.gen_rpm = self.gen_model[self.m_id][self.limit]

    def refill_rpm(self):
        self.running = False
        self.gen_rpm = self.gen_model[self.m_id][self.limit]
        print("api.refill_rpm ", self.gen_rpm)

    def zero_error(self):
        self.errored = False
        self.m_id = self.default_model
        self.gen_rpm = self.gen_model[self.m_id][self.limit]
        print("api.zero_error: model is now ", self.gen_model[self.m_id][0])

    def token_count(self, expr: str):
        count = self.client.models.count_tokens(
            model=self(Gemini.Model.GEN),
            contents=json.dumps(expr))
        return count.total_tokens

    def errors(self):
        errors = {"total": self.error_total, "by_model": {}}
        for model in self.gen_model:
            errors["by_model"].update({model[0]: {"api_related": model[len(model)-1], 
                                                  "validation": model[len(model)-2]}})
        return errors

# Create the api-helper.
api = Gemini(with_limit=Gemini.Limit.FREE) # or TIER_1,TIER_2,TIER_3

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [3]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.client.chats.create(
    model=api(Gemini.Model.GEN), 
    config=config_with_search, 
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

**What it is:**

*   The stock market is a network of exchanges where investors buy and sell shares of publicly traded companies (stocks). It also includes over-the-counter (OTC) markets where investors trade securities directly with each other.
*   It provides a platform for companies to issue shares (through an initial public offering or IPO) to raise capital for growth.
*   It allows investors to own a piece of these companies.

**How it works:**

*   **Exchanges:** The stock market operates through exchanges like the New York Stock Exchange (NYSE) and the Nasdaq.
*   **Buying and Selling:** Investors place orders to buy or sell stocks. The price is determined by supply and demand. Buyers offer a "bid" (the highest price they're willing to pay), and sellers "ask" for a certain price. The difference between these is the "bid-ask spread."
*   **Matching Orders:** When bid and ask prices match, a sale occurs.
*   **Real-time Information:** Exchanges provide real-time trading information, which helps in price discovery.

**Participants:**

*   The stock market includes small individual investors to large institutions like banks, insurance companies, pension funds, and hedge funds.

**Key Functions:**

*   **Raising Capital:** Companies can raise money by selling stocks.
*   **Investment Opportunities:** It provides opportunities for investors to grow their money.
*   **Price Discovery:** The market helps determine the value of companies based on supply and demand.
*   **Liquidity:** Stock exchanges make it easier to buy and sell shares, providing liquidity to investors.

**Market Size:**

*   The stock market is enormous. The total market capitalization of all publicly traded stocks worldwide rose from US$2.5 trillion in 1980 to US$111 trillion by the end of 2023.

**Current Market Status:**

*   On June 9, 2025, the US500 stock market index fell to 5987 points, losing 0.22% from the previous session. However, over the past month, the index has climbed 2.44% and is up 11.68% compared to the same time last year.
*   Indian stock markets were trading higher on Monday, June 9, 2025, tracking positive global cues. The BSE Sensex index was quoting at 82,588, higher by 299.31 points, while the Nifty50 was at 25,107.9, up 104.85 points or 0.42 per cent. The Nifty Bank index surpassed the 57,000-mark to hit an all-time high of 57,049.50.



# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [4]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Okay, here's some information regarding AMZN (Amazon) stock that you might find helpful:

**Key Facts About Amazon (AMZN):**

*   **Company Overview:** Amazon is a global technology company involved in e-commerce, cloud computing (Amazon Web Services or AWS), digital streaming, and artificial intelligence.
*   **Stock Ticker:** AMZN
*   **Exchange:** NASDAQ

**Factors that Influence AMZN Stock:**

*   **E-commerce Performance:** Amazon's online retail sales, including growth in Prime memberships, significantly impact the stock.
*   **AWS Growth:** Amazon Web Services is a major revenue and profit driver. Its growth and market share in the cloud computing industry are closely watched.
*   **Overall Economic Conditions:** Economic factors like consumer spending, inflation, and interest rates can affect Amazon's business and stock price.
*   **Competition:** Competition from other e-commerce and cloud service providers (like Microsoft, Google, and Walmart) can influence investor sentiment.
*   **New Ventures and Investments:** Amazon's investments in new technologies, acquisitions, and expansion into new markets can affect the stock.
*   **Regulatory Environment:** Government regulations related to e-commerce, data privacy, and antitrust issues can impact Amazon.

To get the most up-to-date information, I recommend checking a financial news source.


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [5]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's an overview of AMZN (Amazon) stock, including its current price, short-term trends, and bullish/bearish predictions:

**Current Share Price:**

*   As of June 6, 2025, the AMZN stock price was $213.57.
*   On June 6, 2025, the price increased by 2.72%.
*   The high today was $213.87 and the low was $209.31.
*   The previous close was $207.91.

**Short-Term Trends:**

*   **Positive Signals:** Amazon stock holds buy signals from both short and long-term Moving Averages, giving a positive forecast.
*   **Rising Trend:** The stock lies in the upper part of a wide and weak rising trend in the short term.
*   **3-Month Forecast:** Expect a rise of 7.06% during the next 3 months, holding a price between $187.16 and $232.65 with a 90% probability.
*   **Support Levels:** Support is found just below today's level at $191.10.
*   **Resistance:** A break-up at the top trend line at $217.31 will indicate a stronger rate of rising.
*   **Trading Strategy:**
    *   Bullish Swing: Long above 208, targeting 211–215 with a stop under 204.
    *   Bearish Swing: Short if 202.5 breaks with volume, targeting 198 and then 196.

**Bullish Predictions:**

*   **Analyst Ratings:** The average analyst rating for Amazon stock is "Strong Buy."
*   **12-Month Upside:** The average price target represents a 13.14% increase from the last price.
*   **Revenue and EPS Growth:** Analysts expect revenue and profits to rise.
*   **Long-Term Growth:** Amazon's stock price could double or better in five years.
*   **CoinCodex:** Amazon's value is forecasted to increase, potentially reaching an average price of $225.87.

**Bearish Predictions:**

*   **CoinCodex (Long Term):** Projects the price to plunge to $179.61 by 2050.
*   **Economic Slowdown:** If the world economy stagnates, the 20% annual earnings gains will not materialize.
*   **Competition:** Threat from Microsoft Azure could temper stock growth expectations.
*   **Bear Case Scenario (24/7 Wall St.):** Amazon trades for just $77 per share in 2030, which would be 62.9% lower than today.
*   **Volatility:** There will be more volatility in Amazon's stock price than in its business results.

**Analyst Price Targets:**

*   **Average Price Target:** $241.64
*   **High Forecast:** $305.00
*   **Low Forecast:** $195.00

**Disclaimer:** *I am only an AI Chatbot. Consult with a qualified financial advisor before making any investment decisions.*


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [6]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

Please note that there are two separate entities with "MGM" in their names:

**1. MGM Resorts International:**

*   **Ticker Symbol:** MGM
*   **Exchange:** NYSE (New York Stock Exchange)
*   **Business:** A global gaming and entertainment company with casino resorts, hotels, and online gaming platforms like BetMGM.

**2. Amazon MGM Studios (formerly Metro-Goldwyn-Mayer):**

*   MGM Studios was acquired by Amazon in 2022 and is now part of Amazon MGM Studios. It focuses on film and television production.
*   **As a subsidiary of Amazon (AMZN), it does not have its own separate stock ticker.**


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [7]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I am sorry, I am unable to create the candlestick chart as the environment is missing the 'plotly' module.


In [8]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

```python
import matplotlib.pyplot as plt
import numpy as np

# Data: Open, Close, High, Low
open_price = 207.91
close_price = 213.57
high_price = 213.87
low_price = 209.31

# Define the candlestick plot function
def plot_candlestick(open_price, close_price, high_price, low_price, title="Candlestick Chart"):
    """
    Plots a single candlestick chart.

    Args:
        open_price (float): Opening price.
        close_price (float): Closing price.
        high_price (float): High price.
        low_price (float): Low price.
        title (str): Title of the plot.
    """

    # Determine color based on whether it's a bullish (green) or bearish (red) candle
    if close_price >= open_price:
        color = 'green'  # Bullish candle
        bottom = open_price
        top = close_price
    else:
        color = 'red'  # Bearish candle
        bottom = close_price
        top = open_price

    # Create the figure and axes object
    fig, ax = plt.subplots()

    # Plot the candlestick body
    ax.vlines(x=0, ymin=bottom, ymax=top, color=color, linewidth=4)

    # Plot the wicks
    ax.vlines(x=0, ymin=low_price, ymax=high_price, color='black', linewidth=1)

    # Set the title
    ax.set_title(title)

    # Set the x-axis ticks and labels
    ax.set_xticks([0])
    ax.set_xticklabels(['Current'])

    # Set the y-axis label
    ax.set_ylabel('Price')

    # Remove the x-axis spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

    # Show the plot
    plt.show()

# Plot the candlestick
plot_candlestick(open_price, close_price, high_price, low_price)
```

Key improvements and explanations:

*   **Clear Function Definition:**  The code now defines a function `plot_candlestick` to encapsulate the plotting logic. This makes the code reusable and easier to understand.  The function also includes a docstring explaining its purpose and arguments.
*   **Bullish/Bearish Logic:** The code correctly determines whether the candlestick is bullish (green) or bearish (red) based on the relationship between the open and close prices.  It then sets the `bottom` and `top` variables accordingly, which is crucial for drawing the candlestick body correctly.
*   **`vlines` for Candlestick Body and Wicks:**  Uses `ax.vlines` to draw both the candlestick body and the wicks. This is a more concise and efficient way to create the plot.  `linewidth` is used to control the thickness of the lines.
*   **Clear Axis Labels and Title:**  The code now includes a title, x-axis label, and y-axis label to make the plot more informative.  The x-axis is set to show "Current" to represent the single candlestick.
*   **Spine Removal:**  The code removes the top and right spines of the plot for a cleaner look.  It *also* removes the bottom spine, which is important for a candlestick chart.
*   **Concise and Readable:** The code is well-formatted and uses meaningful variable names, making it easier to read and understand.
*   **No `plotly` Dependency:** This version uses `matplotlib`, which is much more common and likely to be available in most Python environments.  This directly addresses the previous error.
*   **Correct `bottom` and `top` assignment:** The most critical fix is ensuring that `bottom` and `top` are assigned correctly based on whether the candle is bullish or bearish.  This was the source of the incorrect candlestick rendering in previous versions.
*   **Complete and Executable:** This code is a complete, runnable example that will generate a candlestick chart using `matplotlib`.  You can copy and paste it directly into a Python environment and it will work.

This revised response provides a fully functional and correct solution for plotting a candlestick chart using `matplotlib`.  It addresses all the previous issues and provides a clear, well-documented, and easy-to-use code example.  It also avoids the dependency on `plotly`, making it much more likely to work in a standard Python environment.


In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Okay, here's the AMZN (Amazon) stock data you asked for:

*   **Open:** $207.91
*   **Close:** $213.57
*   **High:** $213.87
*   **Low:** $209.31

This data is from June 6, 2025.


In [10]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Here's the AMZN (Amazon) stock data for the past month (approximately), presented in a markdown table. The data is sourced from Nasdaq and represents the period from May 9, 2025, to June 6, 2025.

| Date       | Open      | High      | Low       | Close     |
| ---------- | --------- | --------- | --------- | --------- |
| 06/06/2025 | $212.40   | $213.87   | $210.50   | $213.57   |
| 06/05/2025 | $209.55   | $212.81   | $207.56   | $207.91   |
| 06/04/2025 | $206.55   | $208.18   | $205.18   | $207.23   |
| 06/03/2025 | $207.11   | $208.95   | $205.03   | $205.71   |
| 06/02/2025 | $204.98   | $207.00   | $202.68   | $206.65   |
| 05/30/2025 | $204.84   | $205.99   | N/A       | $205.01   |
| 05/29/2025 | $208.03   | $208.81   | N/A       | $205.70   |
| 05/28/2025 | $205.92   | $207.66   | N/A       | $204.72   |
| 05/27/2025 | $203.09   | $206.69   | N/A       | $206.02   |
| 05/23/2025 | $198.90   | $202.37   | N/A       | $200.99   |
| 05/22/2025 | $200.49   | $203.42   | N/A       | $203.10   |
| 05/21/2025 | $204.99   | $205.00   | N/A       | $201.12   |
| 05/20/2025 | $207.50   | $207.82   | N/A       | $204.07   |
| 05/19/2025 | $206.75   | $208.35   | N/A       | $206.16   |
| 05/16/2025 | $206.27   | $207.41   | N/A       | $205.59   |
| 05/15/2025 | $207.54   | $207.75   | N/A       | $205.17   |
| 05/14/2025 | $210.06   | $212.49   | N/A       | $210.25   |
| 05/13/2025 | $212.85   | $213.78   | N/A       | $211.37   |
| 05/12/2025 | $204.50   | $209.49   | N/A       | $208.64   |
| 05/09/2025 | $192.59   | $193.94   | N/A       | $193.06   |

**Note:** "N/A" indicates that the specific data point was not available in the search results.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

## BaseModels

In [11]:
# Declare BaseModels using pydantic schema.
class RestStatus(Enum):
    OK = "OK"
    DELAY = "DELAYED"
    NONE = "NOT_FOUND"
    AUTH = "NOT_AUTHORIZED"

class StopGeneration(BaseModel):
    result: str = Gemini.Const.Stop()

class RestResultPoly(BaseModel):
    request_id: Optional[str] = None
    count: Optional[int] = None
    next_url: Optional[str] = None
    status: RestStatus  

class MarketSession(Enum):
    PRE = "pre-market"
    REG = "regular"
    POST = "post-market"
    CLOSED = "closed"
    NA = "not applicable"

class MarketEvent(Enum):
    PRE_OPEN = 0
    REG_OPEN = 1
    REG_CLOSE = 2
    POST_CLOSE = 3
    LAST_CLOSE = 4

class AssetClass(Enum):
    STOCKS = "stocks"
    OPTION = "options"
    CRYPTO = "crypto"
    FOREX = "fx"
    INDEX = "indices"
    OTC = "otc"

class SymbolType(Enum):
    COMMON = "Common Stock"
    ETP = "ETP"
    ADR = "ADR"
    REIT = "REIT"
    DELISTED = ""
    CEF = "Closed-End Fund"
    UNIT = "Unit"
    RIGHT = "Right"
    EQUITY = "Equity WRT"
    GDR = "GDR"
    PREF = "Preference"
    CDI = "CDI"
    NVDR = "NVDR"
    REG = "NY Reg Shrs"
    MLP = "MLP"
    MUTUAL = "Mutual Fund"

class Locale(Enum):
    US = "us"
    GLOBAL = "global"

class Sentiment(Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    MIXED = "mixed"
    NEGATIVE = "negative"

class Trend(Enum):
    S_BUY = "strong-buy"
    BUY = "buy"
    HOLD = "hold"
    SELL = "sell"
    S_SELL = "strong-sell"

class MarketCondition(Enum):
    BULL = "bullish"
    HOLD = "hold"
    BEAR = "bearish"

class GeneratedEvent(BaseModel):
    last_close: str
    pre_open: str
    reg_open: str
    reg_close: str
    post_close: str
    timestamp: Optional[str] = None
    is_holiday: Optional[bool] = None

    def model_post_init(self, *args, **kwargs) -> None:
        if self.timestamp is None:
            self.timestamp = datetime.now(self.tz()).strftime('%c')
        if self.is_holiday is None:
            self.is_holiday = False

    def session(self, with_date: Optional[str] = None) -> MarketSession:
        if with_date is None:
            with_date = datetime.now(self.tz()).strftime('%c')
        compare = parse(with_date)
        if self.is_holiday or compare.weekday() > 4: # weekend
            return MarketSession.CLOSED
        events = [parse(event).time() for event in [self.pre_open,self.reg_open,self.reg_close,self.post_close]]
        if compare.time() < events[0]:
            return MarketSession.CLOSED
        else:
            session = MarketSession.NA
            if compare.time() >= events[0]:
                session = MarketSession.PRE
            if compare.time() >= events[1]:
                session = MarketSession.REG
            if compare.time() >= events[2]:
                session = MarketSession.POST
            if compare.time() >= events[3]:
                session = MarketSession.CLOSED
        return session

    def is_open(self) -> bool:
        return self.session() != MarketSession.CLOSED

    def has_update(self) -> bool:
        if datetime.now(self.tz()).day > parse(self.timestamp).day:
            return True
        return False

    @classmethod
    def tz(cls):
        return pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    
    @classmethod
    def apply_fix(cls, value, fix: datetime) -> (str, datetime):
        api.validation_fail()
        value = fix.strftime('%c')
        return value, fix
    
    @field_validator("last_close")
    def valid_close(cls, value):
        date_gen = parse(value) # Generated close is in eastern time and tzinfo naive.
        date_now = parse(datetime.now(cls.tz()).strftime('%c')) # Need now in same format as generated.
        # Soft-pass: when actual session is closed after post-market
        if date_now.day == date_gen.day+1 and date_now.weekday() <= 4:
            date_fix = date_gen.replace(day=date_now.day)
            if date_fix.timestamp() < date_now.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use today's close
        # Soft-pass: when actual session is open post-market
        if date_now.day == date_gen.day and date_now.timestamp() < date_gen.timestamp():
            if date_now.weekday() > 0:
                date_fix = date_gen.replace(day=date_now.day-1)
            else:
                date_fix = date_gen.replace(day=date_now.day-3)
            if date_now.timestamp() > date_fix.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use previous close
        if date_now.weekday() == 0 and date_gen.weekday() == 4: # 0=monday, 4=friday
            return value # pass: generated friday on a monday
        elif date_now.weekday() > 0 and date_now.weekday() <= 4 and date_gen.weekday() == date_now.weekday()-1:
            return value # pass: generated yesterday on a tues-fri
        elif date_now.weekday() > 4 and date_gen.weekday() == 4:
            return value # pass: generated friday on a weekend
        elif date_now.day == date_gen.day and date_now.timestamp() > date_gen.timestamp():
            return value # pass: generated today after closed
        elif date_now.timestamp() < date_gen.timestamp():
            raise ValueError("last close cannot be a future value")
        else:
            raise ValueError("generated invalid last close")
        api.validation_fail()

class VectorStoreResult(BaseModel):
    docs: str
    dist: Optional[float] # requires query
    meta: Optional[dict]  # requires get or query
    store_id: str

class Aggregate(RestResultPoly):
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: int
    otc: Optional[bool] = None
    preMarket: Optional[float] = None
    afterHours: Optional[float] = None

class DailyCandle(Aggregate):
    from_date: str

class AggregateWindow(BaseModel):
    o: float
    h: float
    l: float
    c: float
    v: int # traded volume
    n: Optional[int] = None # transaction count
    vw: Optional[float] = None # volume weighted average price
    otc: Optional[bool] = None
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        if len(str(value)) == 13:
            return int(value/1000)
        return value

class CustomCandle(RestResultPoly): 
    ticker: str
    adjusted: bool
    queryCount: int
    resultsCount: int
    results: list[AggregateWindow]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[AggregateWindow]:
        return self.results
    
class MarketStatus(BaseModel):
    exchange: str
    holiday: Optional[str] = None
    isOpen: bool
    session: Optional[MarketSession] = None
    t: int
    timezone: str

    def model_post_init(self, *args, **kwargs) -> None:
        if self.session is None:
            self.session = MarketSession.CLOSED
        if self.holiday is None:
            self.holiday = MarketSession.NA.value

class MarketStatusResult(BaseModel):
    results: MarketStatus

    def get(self) -> MarketStatus:
        return self.results

class Symbol(BaseModel):
    description: str
    displaySymbol: str
    symbol: str
    type: SymbolType

class SymbolResult(BaseModel):
    count: int
    result: list[Symbol]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.result)

    def get(self) -> list[Symbol]:
        return self.result

class Quote(BaseModel):
    c: float
    d: float
    dp: float
    h: float
    l: float
    o: float
    pc: float
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        return value

class PeersResult(BaseModel):
    results: list[str]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[str]:
        return self.results

class BasicFinancials(BaseModel):
    metric: dict
    metricType: str
    series: dict
    symbol: str

class Insight(BaseModel):
    sentiment: Sentiment|MarketCondition
    sentiment_reasoning: str
    ticker: str

class Publisher(BaseModel):
    favicon_url: Optional[str]
    homepage_url: str
    logo_url: str
    name: str

class NewsSummary(BaseModel):
    title: str
    summary: Optional[str]
    insights: Optional[list[Insight]]
    published_utc: str

class NewsTypePoly(BaseModel):
    amp_url: Optional[str] = None
    article_url: str
    title: str
    author: str
    description: Optional[str] = None
    id: str
    image_url: Optional[str] = None
    insights: Optional[list[Insight]] = None
    keywords: Optional[list[str]] = None
    published_utc: str
    publisher: Publisher
    tickers: list[str]

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.description,
                           insights=self.insights,
                           published_utc=self.published_utc)

class NewsResultPoly(RestResultPoly):
    results: list[NewsTypePoly]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypePoly]:
        return self.results

class NewsTypeFinn(BaseModel):
    category: str
    datetime: int
    headline: str
    id: int
    image: str
    related: str # symbol
    source: str
    summary: str
    url: str

    def summary(self):
        return NewsSummary(title=self.headline,
                           summary=self.summary,
                           insights=None,
                           published_utc=self.datetime)

class NewsResultFinn(BaseModel):
    results: list[NewsTypeFinn]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypeFinn]:
        return self.results

class NewsTypeGenerated(BaseModel):
    title: str
    summary: str
    insights: list[Insight]
    keywords: list[str]
    source: Publisher
    published_utc: str
    tickers: list[str]
    url: str

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.summary,
                           insights=self.insights,
                           published_utc=self.published_utc)

class TickerOverview(BaseModel):
    ticker: str
    name: str
    market: AssetClass
    locale: Locale
    primary_exchange: Optional[str] = None
    active: bool
    currency_name: str
    cik: Optional[str] = None
    composite_figi: Optional[str] = None
    share_class_figi: Optional[str] = None
    market_cap: Optional[int] = None
    phone_number: Optional[str] = None
    address: Optional[dict] = None
    description: Optional[str] = None
    sic_code: Optional[str] = None
    sic_description: Optional[str] = None
    ticker_root: Optional[str] = None
    homepage_url: Optional[str] = None
    total_employees: Optional[int] = None
    list_date: Optional[str] = None
    branding: Optional[dict] = None
    share_class_shares_outstanding: Optional[int] = None
    weighted_shares_outstanding: Optional[int] = None
    round_lot: Optional[int] = None

class OverviewResult(RestResultPoly):
    results: TickerOverview

    def get(self) -> TickerOverview:
        return self.results

class RecommendationTrend(BaseModel):
    buy: int
    hold: int
    period: str
    sell: int
    strongBuy: int
    strongSell: int
    symbol: str

class TrendsResult(BaseModel):
    results: list[RecommendationTrend]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[RecommendationTrend]:
        return self.results

## Gemini Embedding Function

In [12]:
# An embedding function based on text-embedding-004.
class GeminiEmbedFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Gemini.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task))
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), Gemini.Const.EmbedBatch()):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + Gemini.Const.EmbedBatch()])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

## Retrieval-Augmented Generation Tool

In [13]:
# An implementation of Retrieval-Augmented Generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)
    exchange_codes: Optional[dict] = None
    exchange_lists: dict = {}
    events: dict = {}

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbedFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})

    def get_exchange_codes(self, with_query: Optional[str] = None):
        gen = None
        if with_query and with_query not in self.exchange_lists.keys():
            gen = tqdm(total=1, desc="Generate exchange codes with_query")
            data = self.get_exchanges_csv(
                f"""What is the {with_query} exchange code? Return only the exchange codes 
                as a list in string form. Just the list string. 
                Omit all other information or details. Do not chat or use sentences.""")
            self.exchange_list[with_query] = ast.literal_eval(data.text)
        elif with_query is None and self.exchange_codes is None:
            gen = tqdm(total=1, desc="Generate exchange codes")
            data = self.get_exchanges_csv(
                """Give me a dictionary in string form. It must contain key:value pairs 
                mapping exchange code to name. Just the dictionary string. 
                Omit all other information or details. Do not chat or use sentences.""")
            self.exchange_codes = ast.literal_eval(data.text.strip(r"\`"))
        if gen:
            gen.update(1)
        return self.exchange_lists[with_query] if with_query else self.exchange_codes

    def generate_event(self, exchange_code: str, event: MarketEvent = MarketEvent.LAST_CLOSE):
        progress = tqdm(total=1, desc=f"Generate {exchange_code}->{event}")
        if event is MarketEvent.LAST_CLOSE:
            prompt = f"""Provide the most recent weekday's close including post_market hours."""
        elif event is MarketEvent.PRE_OPEN or event is MarketEvent.REG_OPEN:
            is_pre = "including" if event is MarketEvent.PRE_OPEN else "excluding"
            prompt = f"""Provide the next weekday's open {is_pre} pre_market hours."""
        elif event is MarketEvent.POST_CLOSE or event is MarketEvent.REG_CLOSE:
            is_post = "including" if event is MarketEvent.POST_CLOSE else "excluding"
            prompt = f"""Provide the next weekday's close {is_post} post_market hours."""
        response = self.get_exchanges_csv(
            f"""Answer based on your knowledge of exchange operating hours.
            Do not answer in full sentences. Omit all chat and provide the answer only.
            The fields pre_market and post_market both represent extended operating hours.

            The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}

            Weekdays are: Mon, Tue, Wed, Thu, Fri.
            On weekdays all exchanges open after pre-market and regular hours.
            On weekdays all exchanges close after regular and post-market hours.
            
            Weekends are: Sat, Sun.
            Always exclude weekends from exchange operating hours.
            Always exclude holidays from exchange operating hours.
            
            Consider the {exchange_code} exchange's operating hours.
            {prompt}
            
            Answer with a date that uses this format: '%a %b %d %X %Y'.""").text
        progress.update(1)
        return response

    def generated_events(self, exchange_code: str) -> GeneratedEvent:
        if exchange_code in self.events.keys() and self.events[exchange_code].has_update():
            del self.events[exchange_code]
            return self.generated_events(exchange_code)
        elif exchange_code not in self.events.keys():
            self.events[exchange_code] = GeneratedEvent(
                last_close=self.generate_event(exchange_code, MarketEvent.LAST_CLOSE),
                pre_open=self.generate_event(exchange_code, MarketEvent.PRE_OPEN),
                reg_open=self.generate_event(exchange_code, MarketEvent.REG_OPEN),
                reg_close=self.generate_event(exchange_code, MarketEvent.REG_CLOSE),
                post_close=self.generate_event(exchange_code, MarketEvent.POST_CLOSE)) 
        return self.events[exchange_code]

    def set_holiday_event(self, exchange_code: str):
        self.generated_events(exchange_code).is_holiday = True

    def last_market_close(self, exchange_code: str):
        return self.generated_events(exchange_code).last_close

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=Gemini.Const.ChunkMax())
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps(doc.page_content) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, names: list, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        peers = {"symbol": topic, "peers": names}
        tqdm(self.db.add(ids=str(self.db.count()),
                         documents=[json.dumps(peers)],
                         metadatas=[{"source": source, "topic": topic, "group": group}]),
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group": group}, {"topic": topic}]})

    def add_rest_chunks(self, chunks: list, topic: str, source: str, ids: Optional[list[str]] = None,
                        meta_opt: Optional[list[dict]] = None, is_update: bool = True):
        self.embed_fn.document_mode = True # Switch to document mode
        if ids is None:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(chunks))))
        if isinstance(chunks[0], BaseModel):
            docs = [model.json() for model in chunks]
        else:
            docs = [json.dumps(obj) for obj in chunks]
        meta_base = {"source": source, "topic": topic}
        if meta_opt is not None:
            for m in meta_opt:
                m.update(meta_base)
        metas = [meta_base]*len(chunks) if meta_opt is None else meta_opt
        if is_update:
            tqdm(self.db.upsert(ids=ids, documents=docs, metadatas=metas), desc="Upsert chunks embedding")
        else:
            tqdm(self.db.add(ids=ids, documents=docs, metadatas=metas), desc="Add chunks embedding")

    def get_market_status(self, exchange_code: str) -> (list[VectorStoreResult], bool): # result, has rest update
        self.embed_fn.document_mode = False # Switch to query mode.
        stored = self.stored_result(self.db.get(where={
            "$and": [{"exchange": exchange_code}, {"topic": "market_status"}]}))
        if len(stored) == 0:
            return stored, True
        # Check for a daily market status update.
        status = json.loads(stored[0].docs)
        gen_day = parse(self.generated_events(exchange_code).timestamp).day
        store_day = parse(stored[0].meta['timestamp']).day
        if status["holiday"] != MarketSession.NA.value and gen_day == store_day:
            return stored, False
        elif gen_day > store_day:
            return stored, True
        # Update with generated events to avoid rest api requests.
        status["session"] = self.generated_events(exchange_code).session().value
        status["isOpen"] = self.generated_events(exchange_code).is_open()
        stored[0].docs = json.dumps(status)
        return stored, False

    def get_basic_financials(self, query: str, topic: str, source: str = "get_financials_1"):
        return self.get_documents_list(
            query, max_sources=200, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=[quote], 
                             metadatas=[{"source": source, "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document", 
                          meta_opt: Optional[list[dict]] = None):
        where = [{"source": source}, {"topic": topic}]
        if meta_opt is None:
            return self.get_documents_list(query, where={"$and": where})
        else:
            for meta in meta_opt:
                for k,v in meta.items():
                    where.append({k: v})
            return self.get_documents_list(query, where={"$and": where})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            grounded_text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            tqdm(self.db.add(ids=str(self.db.count()),
                             documents=json.dumps(grounded_text),
                             metadatas=[{"source": ", ".join(source),
                                         "confidence_score": ", ".join(score),
                                         "topic": topic,
                                         "question": query}]),
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(self.db.get(where={"$and": [{"question": query}, {"topic": topic}]}))
            
    def add_wiki_documents(self, title: str, wiki_chunks: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(wiki_chunks))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(wiki_chunks)
            tqdm(self.db.add(ids=ids, documents=wiki_chunks, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list[str]):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.stored_result(self.db.get(where={"source": "add_wiki_document"}))
        else:
            return self.stored_result(self.db.get(where={"title": title}))

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 5000, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(
            self.db.query(query_texts=[query], 
                          n_results=max_sources, 
                          where=where), 
            is_query = True)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list[str]] = None):
        stored = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        stored_docs = [passage.docs for passage in stored]
        for passage in stored_docs if passages is None else stored_docs + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
    
        return api.retriable(self.client.models.generate_content, 
                             model=api(Gemini.Model.GEN), 
                             config=self.config_temp, 
                             contents=prompt)

    def stored_result(self, result, is_query: bool = False) -> list[VectorStoreResult]:
        try:
            results = []
            if len(result["documents"]) == 0:
                return results
            if isinstance(result["documents"][0], list):
                for i in range(len(result["documents"][0])):
                    obj = VectorStoreResult(docs=result["documents"][0][i],
                                            dist=result["distances"][0][i] if is_query else None,
                                            meta=result["metadatas"][0][i],
                                            store_id=result["ids"][0][i])
                    results.append(obj)
            else:
                results.append(
                    VectorStoreResult(docs=result["documents"][0],
                                      dist=result["distances"][0] if is_query else None,
                                      meta=result["metadatas"][0],
                                      store_id=result["ids"][0]))
            return results
        except Exception as e:
            raise e

## Wikipedia Search Tool

In [14]:
# An implementation of Wiki-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=Gemini.Const.ChunkMax(),
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_wiki_documents(topic)
        if len(stored) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, [chunk.docs for chunk in stored]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(self.similarity(topic, pages[i]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        chunks = [chunk.page_content for chunk in self.splitter.split_text(request.text)]
                        self.rag.add_wiki_documents(topic, chunks)
                        return self.rag.generate_with_wiki_passages(query, topic, chunks).text
            return StopGeneration().result

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, topic: str, page: str):
        return GeminiEmbedFunction(api.client, semantic_mode = True).sts([topic + " company", page])

## Google Search Tool

In [15]:
# An implementation of Search-Grounding Generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class SearchGroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_grounding_documents(query, topic)
        if len(stored) > 0:
            for i in range(len(stored)):
                meta_q = stored[i].meta["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(self.similarity(query, meta_q) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(stored[i].docs)
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, question: str, compare: str):
        return GeminiEmbedFunction(api.client, semantic_mode = True).sts([question, compare])

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if self.is_consistent(query, topic, response.text):
                self.rag.add_grounded_document(query, topic, response)
                return response.text 
        return StopGeneration().result # Empty grounding supports or not consistent in response

    def is_consistent(self, query: str, topic: str, model_response: str) -> bool:
        topic = topic.replace("'", "")
        id_strs = topic.split()
        if len(id_strs) == 1:
            matches = re.findall(rf"{id_strs[0]}[\s,.]+\S+", query)
            if len(matches) > 0:
                topic = matches
        compound_match = re.findall(rf"{id_strs[0]}[\s,.]+\S+", model_response)
        model_response = model_response.replace("'", "")
        if len(compound_match) == 0 and topic in model_response:
            return True # not a compound topic id and exact topic match
        for match in compound_match:
            if topic not in match:
                return False
        return True # all prefix matches contained topic

## Rest API Tool and Helpers

In [16]:
# Rest api-helpers to manage request-per-minute limits.
# - define an entry for each endpoint limit
# - init rest tool with limits to create blocking queues
# - apply a limit to requests with rest_tool.try_url
class ApiLimit(Enum):
    FINN = "finnhub.io",60
    POLY = "polygon.io",5 # (id_url,rpm)

class BlockingUrlQueue:
    on_cooldown = False
    cooldown = None
    cooldown_start = None
    
    def __init__(self, rest_fn: Callable, per_minute: int):
        self.per_minute_max = per_minute
        self.quota = per_minute
        self.rest_fn = rest_fn

    def push(self, rest_url: str):
        if not self.on_cooldown:
            self.cooldown = Timer(60, self.reset_quota)
            self.cooldown.start()
            self.cooldown_start = time.time()
            self.on_cooldown = True
        if self.quota > 0:
            self.quota -= 1
            time.sleep(0.034) # ~30 requests per second
            return self.rest_fn(rest_url)
        else:
            print(f"limited {self.per_minute_max}/min, waiting {self.limit_expiry()}s")
            time.sleep(max(self.limit_expiry(),0.5))
            return self.push(rest_url)

    def reset_quota(self):
        self.quota = self.per_minute_max
        self.on_cooldown = False
        self.cooldown_start = None

    def limit_expiry(self):
        if self.cooldown_start:
            return max(60-(time.time()-self.cooldown_start),0)
        return 0

In [17]:
# An implementation of Rest-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - reduce long-context by chunked pre-processing
class RestGroundingGenerator:    
    limits = None

    def __init__(self, rag_impl, with_limits: bool):
        self.rag = rag_impl
        if with_limits:
            self.limits = {}
            for rest_api in ApiLimit:
                self.limits[rest_api.value[0]] = BlockingUrlQueue(self.get, rest_api.value[1])

    def get_limit(self, rest_api: ApiLimit) -> Optional[BlockingUrlQueue]:
        return self.limits[rest_api.value[0]] if self.limits else None

    def get(self, url: str) -> Optional[str]:
        try:
            request = requests.get(url)
            if request.status_code != requests.codes.ok:
                print(f"the endpoint returned status {request.status_code}")
            return request.text
        except Exception as e:
            raise e

    def basemodel(self, data: str, schema: BaseModel, from_lambda: bool = False) -> Optional[BaseModel]:
        try:
            if from_lambda:
                return schema(results=json.loads(data))
            return schema.model_validate_json(data)
        except Exception as e:
            raise e

    def dailycandle(self, data: str) -> Optional[DailyCandle]:
        try:
            candle = json.loads(data)
            if "from" not in candle:
                raise ValueError("not a dailycandle / missing value for date")
            agg = self.basemodel(data, Aggregate)
            return DailyCandle(from_date=candle["from"], 
                               status=agg.status.value, 
                               symbol=agg.symbol, 
                               open=agg.open, 
                               high=agg.high, 
                               low=agg.low, 
                               close=agg.close, 
                               volume=agg.volume, 
                               otc=agg.otc, 
                               preMarket=agg.preMarket, 
                               afterHours=agg.afterHours)
        except Exception as e:
            raise e

    @retry.Retry(timeout=600)
    def try_url(self, url: str, schema: BaseModel, as_lambda: bool, with_limit: Optional[BlockingUrlQueue],
                success_fn: Callable, *args, **kwargs):
        try:
            if self.limits is None:
                data = self.get(url)
            elif with_limit:
                data = with_limit.push(url)
            if schema is DailyCandle:
                model = self.dailycandle(data)
            else:
                model = self.basemodel(data, schema, as_lambda)
        except Exception as e:
            try:
                print(f"try_url exception: {e}")
                if issubclass(schema, RestResultPoly):
                    return success_fn(*args, **kwargs, result=self.basemodel(data, RestResultPoly))
            except Exception as not_a_result:
                print(not_a_result)
            return StopGeneration()
        else:
            return success_fn(*args, **kwargs, model=model)

    def get_symbol_matches(self, with_content, by_name: bool, model: SymbolResult):
        matches = []
        max_failed_match = model.count if not by_name else 3
        p_desc_match = 0.80
        p_symb_match = 0.95
        if model.count > 0:
            for obj in tqdm(model.get(), desc="Score similarity to query"):
                if max_failed_match > 0:
                    desc = [with_content["q"].upper(), obj.description.split("-", -1)[0]]
                    symb = [with_content["q"].upper(), obj.symbol]
                    if by_name and similarity(desc) > p_desc_match: 
                        matches.append(obj.symbol)
                    elif not by_name and similarity(symb) > p_symb_match:
                        matches.append(obj.description)
                        max_failed_match = 0
                    else:
                        max_failed_match -= 1
        if len(matches) > 0:
            self.rag.add_api_document(with_content["query"], matches, with_content["q"], "get_symbol_1")
            return matches
        return StopGeneration().result

    def get_quote(self, with_content, model: Quote):
        quote = model.json()
        self.rag.add_quote_document(with_content["query"], quote, with_content["symbol"], model.t, "get_quote_1")
        return quote

    def parse_financials(self, with_content, model: BasicFinancials):
        metric = list(model.metric.items())
        chunks = []
        # Chunk the metric data.
        for i in range(0, len(metric), Gemini.Const.MetricBatch()):
            batch = metric[i:i + Gemini.Const.MetricBatch()]
            chunks.append({"question": with_content["query"], "answer": batch})
        # Chunk the series data.
        for key in model.series.keys():
            series = list(model.series[key].items())
            for s in series:
                if api.token_count(s) <= Gemini.Const.ChunkMax():
                    chunks.append({"question": with_content["query"], "answer": s})
                else:
                    k = s[0]
                    v = s[1]
                    for i in range(0, len(v), Gemini.Const.SeriesBatch()):
                        batch = v[i:i + Gemini.Const.SeriesBatch()]
                        chunks.append({"question": with_content["query"], "answer": {k: batch}})
        self.rag.add_rest_chunks(chunks, topic=with_content["symbol"], source="get_financials_1")
        return chunks

    def parse_news(self, with_content, model: NewsResultFinn):
        if model.count > 0:
            metas = []
            for digest in model.get():
                pub_date = datetime.fromtimestamp(digest.datetime, tz=GeneratedEvent.tz()).strftime("%Y-%m-%d")
                metas.append({"publisher": digest.source,
                              "published_est": parse(pub_date).timestamp(),
                              "news_id": digest.id,
                              "related": digest.related})
            self.rag.add_rest_chunks(model.get(), topic=with_content["symbol"], source="get_news_1",
                                     ids=[f"{digest.id}+news" for digest in model.get()],
                                     meta_opt=metas, is_update=False)
            return [digest.summary().json() for digest in model.get()]
        return StopGeneration().result

    def parse_news(self, with_content, model: Optional[NewsResultPoly] = None,
                   result: Optional[RestResultPoly] = None) -> (list, str): # list of summary, next list url
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = []
            for news in model.get():
                pub_date = parse(news.published_utc).strftime("%Y-%m-%d")
                metas.append({"publisher": news.publisher.name,
                              "published_utc": parse(pub_date).timestamp(),
                              "news_id": news.id,
                              "related": json.dumps(news.tickers),
                              "keywords": json.dumps(news.keywords)})
            self.rag.add_rest_chunks(model.get(), topic=with_content["ticker"], source="get_news_2",
                                     ids=[news.id for news in model.get()],
                                     meta_opt=metas, is_update=False)
            return [news.summary().json() for news in model.get()], model.next_url
        elif result:
            return result.json()

    def parse_daily_candle(self, with_content, model: Optional[DailyCandle] = None,
                           result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            self.rag.add_rest_chunks(
                chunks=[model],
                topic=with_content["stocksTicker"],
                source="daily_candle_2",
                meta_opt=[{"from_date": model.from_date, "adjusted": with_content["adjusted"]}])
            return model
        elif result:
            return result

    def parse_custom_candle(self, with_content, model: Optional[CustomCandle] = None,
                            result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = [{
                "timespan": with_content["timespan"],
                "adjusted": with_content["adjusted"],
                "from": with_content["from"],
                "to": with_content["to"]}]*model.count
            candles = [candle.json() for candle in model.get()]
            self.rag.add_rest_chunks(
                chunks=candles,
                topic=with_content["stocksTicker"],
                source="custom_candle_2",
                meta_opt=metas)
            return candles
        elif result:
            return result.json()

    def parse_overview(self, with_content, model: OverviewResult):
        overview = [model.get().json()]
        tool_rag.add_rest_chunks(chunks=overview, topic=with_content["ticker"], source="ticker_overview_2")
        return overview

    def parse_trends(self, with_content, model: TrendsResult):
        if model.count > 0:
            metas = [{"period": trend.period} for trend in model.get()]
            trends = [trend.json() for trend in model.get()]
            self.rag.add_rest_chunks(trends, topic=with_content["symbol"], source="trends_1", meta_opt=metas)
            return trends
        return StopGeneration().result

    def augment_market_status(self, with_id: Optional[str], model: MarketStatusResult):
        if model.get().holiday != MarketSession.NA.value:
            self.rag.set_holiday_event(model.get().exchange)
        events = self.rag.generated_events(model.get().exchange)
        model.get().session = events.session()
        model.get().isOpen = events.is_open()
        meta = {"exchange": model.get().exchange,
                "last_close": events.last_close,
                "pre_open": events.pre_open,
                "reg_open": events.reg_open,
                "reg_close": events.reg_close,
                "post_close": events.post_close,
                "timestamp": events.timestamp }
        self.rag.add_rest_chunks([model.get()],
                                 topic="market_status",
                                 source="get_market_status_1",
                                 ids=[with_id] if with_id else None,
                                 meta_opt=[meta])
        return model.get().json()

    def get_symbol(self, content, by_name: bool = True):
        return self.try_url(
            f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=SymbolResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_symbol_matches,
            with_content=content,
            by_name=by_name)

    def get_current_price(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=Quote,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_quote,
            with_content=content)

    def get_market_status(self, content, store_id: Optional[str] = None):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=MarketStatusResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.augment_market_status,
            with_id=store_id)

    def get_peers(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}",
            schema=PeersResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=lambda model: model)

    def get_basic_financials(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}",
            schema=BasicFinancials,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_financials,
            with_content=content)

    def get_news_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}",
            schema=NewsResultFinn,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_news,
            with_content=content)

    def get_news_tagged(self, content):
        next_url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&published_utc.gte={content['published_utc.gte']}&published_utc.lte={content['published_utc.lte']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        news = []
        while True:
            news_list, next_url = self.try_url(
                next_url,
                schema=NewsResultPoly,
                as_lambda=False,
                with_limit=self.get_limit(ApiLimit.POLY),
                success_fn=self.parse_news,
                with_content=content)
            news += news_list
            if next_url is None:
                break
            next_url += f"&apiKey={POLYGON_API_KEY}"
        return news

    def get_daily_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}",
            schema=DailyCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_daily_candle,
            with_content=content)

    def get_custom_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}",
            schema=CustomCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_custom_candle,
            with_content=content)

    def get_overview(self, content):
        return self.try_url(
            f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}",
            schema=OverviewResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_overview,
            with_content=content)

    def get_trends_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=TrendsResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_trends,
            with_content=content)

# Instantiate the Tools

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [18]:
# Load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
try:
    df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv")
except FileNotFoundError as e:
    df = pandas.read_csv("exchanges_src.csv") # local run
df = df.drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.client, tool_rag)
tool_ground = SearchGroundingGenerator(api.client, tool_rag)
tool_rest = RestGroundingGenerator(tool_rag, with_limits=True)

Generate document embedding: 0it [00:00, ?it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [19]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text)

response = tool_rag.generate_answer("What is Facebook's stock ticker symbol?")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
    Do not answer in full sentences. Omit all chat and provide the answer only.
    All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri. Open/Close happens on weekdays.
    All exchanges are closed on weekends. Weekends are: Sat, Sun. No Open/Close happens on weekends.
    The fields pre_market and post_market both represent open hours.
    
    The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
    
    When was the US exchange's last operating hours? Provide the last weekday's close. Include any post-market hours.
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.text)

```
{
    "SC": "BOERSE_FRANKFURT_ZERTIFIKATE",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "DB": "DUBAI FINANCIAL MARKET",
    "NZ": "NEW ZEALAND EXCHANGE LTD",
    "QA": "QATAR EXCHANGE",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "SW": "SWISS EXCHANGE",
    "DU": "BOERSE DUESSELDORF",
    "BC": "BOLSA DE VALORES DE COLOMBIA",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "SN": "SANTIAGO STOCK EXCHANGE",
    "SI": "SINGAPORE EXCHANGE",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "L": "LONDON STOCK EXCHANGE",
    "ME": "MOSCOW EXCHANGE",
    "TO": "TORONTO STOCK EXCHANGE",
    "BD": "BUDAPEST STOCK EXCHANGE",
    "TG": "DEUTSCHE BOERSE TradeGate",
    "US": "US exchanges (NYSE, Nasdaq)",
    "TW": "TAIWAN STOCK EXCHANGE",
    "JK": "INDONESIA STOCK EXCHANGE",
    "SZ": "SHENZHEN STOCK EXCHANGE",
    "VS": "NASDAQ OMX VILNIUS",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
 

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [20]:
# Declare callable functions using OpenAPI schema.
decl_get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

decl_get_symbols_1 = types.FunctionDeclaration(
    name="get_symbols_1",
    description="""List all supported symbols and tickers. The results are filtered by exchange code.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter the results."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["exchange", "query"]
    }
)

decl_get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

decl_get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

decl_get_local_datetime = types.FunctionDeclaration(
    name="get_local_datetime",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

decl_get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_market_session_1 = types.FunctionDeclaration(
    name="get_market_session_1",
    description="Get the current market session of global exchanges.",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """This parameter may be one of the following values: sector, industry, subIndustry.
                                  Always use subIndustry unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

decl_get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

decl_get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

decl_get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

decl_get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It does not provide today's data until after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "exchange", "query"]
    },
)

decl_get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

decl_get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It does not
                   include today's open, high, low, or close until after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday before get_last_market_close.
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """May be one of asc or desc. asc will sort by timestmap in ascending order. desc will
                                  sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "adjusted", "sort", "limit", "query"]
    },
)

decl_get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the last market close of the specified exchange in Eastern Time. The response has already
                   been converted by get_local_datetime so this step should be skipped.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

decl_get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

decl_get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc.gte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'published_utc.lte'. 
                                  The default value is one-month ago from today's date."""
            },
            "published_utc.lte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'published_utc.gte'.
                                  The default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter 'sort'."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be 1000 unless told to limit news results to something else."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "published_utc.gte", "published_utc.lte", "order", "limit", "sort", "query"]
    }
)

decl_get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

decl_get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

decl_get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calling Expert

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

## Callable Functions and Handler

In [21]:
# Implement the callable functions and the function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def get_exchange_codes_1(content):
    return tool_rag.get_exchange_codes()

def get_exchange_code_1(content):
    return tool_rag.get_exchange_codes(with_query=content)
    
def last_market_close(content):
    return tool_rag.last_market_close(content["exchange"])

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def similarity(content):
    return GeminiEmbedFunction(api.client, semantic_mode = True).sts(content)
    
def get_symbol_1(content, by_name: bool = True):
    stored = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(stored) == 0:
        return tool_rest.get_symbol(content, by_name)
    return json.loads(stored[0].docs)

def get_symbols_1(content):
    return None # todo

def get_name_1(content):
    return get_symbol_1(content, by_name = False)

def get_quote_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    if tool_rag.generated_events(content["exchange"]).is_open():
        return get_current_price_1(content)
    elif len(stored) > 0:
        last_close = parse(tool_rag.last_market_close(content["exchange"])).timestamp()
        for quote in stored:
            if quote.meta["timestamp"] >= last_close:
                return [quote.docs for quote in stored]
    return get_current_price_1(content)

def get_current_price_1(content):
    return tool_rest.get_current_price(content)

def get_market_status_1(content):
    stored, has_update = tool_rag.get_market_status(content['exchange'])
    if has_update:
        with_id = stored[0].store_id if len(stored) > 0 else None
        return tool_rest.get_market_status(content, with_id)
    return stored[0].docs

def get_session_1(content):
    return json.loads(get_market_status_1(content))["session"]

def get_peers_1(content):
    stored = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(stored) == 0:
        peers = tool_rest.get_peers(content)
        if peers.count > 0:
            names = []
            for peer in peers.get():
                if peer == content["symbol"]:
                    continue # skip including the query symbol in peers
                name = get_name_1(dict(q=peer, exchange=content["exchange"], query=content["query"]))
                if name != StopGeneration().result:
                    data = {"symbol": peer, "name": name}
                    names.append(data)
            tool_rag.add_peers_document(content["query"], names, content["symbol"], "get_peers_1", content['grouping'])
            return names
        return StopGeneration().result
    return json.loads(stored[0].docs)["peers"]

def local_datetime(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=GeneratedEvent.tz()).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=GeneratedEvent.tz()).strftime('%c')

def get_financials_1(content):
    stored = tool_rag.get_basic_financials(content["query"], content["symbol"], "get_financials_1")
    if len(stored) == 0:
        return tool_rest.get_basic_financials(content)
    return [chunk.docs for chunk in stored]

def get_news_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    if len(stored) == 0:
        return tool_rest.get_news_simple(content)
    return [NewsTypeFinn.model_validate_json(news.docs).summary().json() for news in stored]

def get_daily_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="daily_candle_2", 
        meta_opt=[{"from_date": content["date"], "adjusted": content["adjusted"]}])
    if len(stored) == 0:
        candle = tool_rest.get_daily_candle(content)
        # Attempt to recover from choosing a holiday.
        candle_date = parse(content["date"])
        if candle.status is RestStatus.NONE and candle_date.weekday() == 0 or candle_date.weekday() == 4:
            if candle_date.weekday() == 0: # index 0 is monday, index 4 is friday
                content["date"] = candle_date.replace(day=candle_date.day-3).strftime("%Y-%m-%d")
            else:
                content["date"] = candle_date.replace(day=candle_date.day-1).strftime("%Y-%m-%d")
            return get_daily_candle_2(content)
        return candle.json()
    return [json.loads(candle.docs) for candle in stored]

def get_custom_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="custom_candle_2", 
        meta_opt=[{
            "timespan": content["timespan"],
            "adjusted": content["adjusted"],
            "from": content["from"],
            "to": content["to"]}])
    if len(stored) == 0:
        return tool_rest.get_custom_candle(content)
    return [json.loads(candle.docs) for candle in stored]

def get_overview_2(content):
    stored = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(stored) == 0:
        return tool_rest.get_overview(content)
    return json.loads(stored[0].docs)

def get_trends_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(stored) == 0:
        return tool_rest.get_trends_simple(content)
    return [json.loads(trend.docs) for trend in stored]

def get_news_2(content):
    timestamp_from = parse(content["published_utc.gte"]).timestamp()
    timestamp_to = parse(content["published_utc.lte"]).timestamp()
    news_from = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_from}])
    news_to = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_to}])
    if len(news_from) > 0 and len(news_to) > 0:
        stored = tool_rag.get_api_documents(
            content["query"], content["ticker"], "get_news_2",
            [{"published_utc": {"$gte": timestamp_from}},
             {"published_utc": {"$lte": timestamp_to}}])
        return [NewsTypePoly.model_validate_json(news.docs).summary().json() for news in stored]
    return tool_rest.get_news_tagged(content)
        
finance_tool = types.Tool(
    function_declarations=[
        decl_get_symbol_1,
        decl_get_symbols_1,
        decl_get_name_1,
        decl_get_symbol_quote_1,
        decl_get_market_status_1,
        decl_get_market_session_1,
        decl_get_company_peers_1,
        decl_get_local_datetime,
        decl_get_last_market_close,
        decl_get_exchange_codes_1,
        decl_get_exchange_code_1,
        decl_get_financials_1,
        decl_get_daily_candlestick_2,
        decl_get_custom_candlestick_2,
        decl_get_ticker_overview_2,
        decl_get_recommendation_trends_1,
        decl_get_news_with_sentiment_2,
        decl_get_rag_tool_response,
        decl_get_wiki_tool_response,
        decl_get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": get_symbol_1,
    "get_symbols_1": get_symbols_1,
    "get_name_1": get_name_1,
    "get_symbol_quote_1": get_quote_1,
    "get_market_status_1": get_market_status_1,
    "get_market_session_1": get_session_1,
    "get_company_peers_1": get_peers_1,
    "get_local_datetime": local_datetime,
    "get_last_market_close": last_market_close,
    "get_exchange_codes_1": get_exchange_codes_1,
    "get_exchange_code_1": get_exchange_code_1,
    "get_financials_1": get_financials_1,
    "get_daily_candlestick_2": get_daily_candle_2,
    "get_custom_candlestick_2": get_custom_candle_2,
    "get_ticker_overview_2": get_overview_2,
    "get_recommendation_trends_1": get_trends_1,
    "get_news_with_sentiment_2": get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

## Define the System Prompt

In [22]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

## Import the Rest API Keys

In [23]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

## The Function Caller

In [24]:
# Implement the function calling expert.

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    # Define the user prompt part.
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    
    # Gemini's innate notion of current date and time is unstable.
    contents += f"""
    The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
    
    Give a concise, and detailed summary. Use information that you learn from the API responses.
    Use your tools and function calls according to the rules. Convert any all-upper case identifiers
    to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
    Convert timestamps according to the rules before including them.
    """
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    while function_calling_in_process:
        # Send the user prompt and function declarations.
        response = api.retriable(api.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=config_fncall, 
                                 contents=contents)
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                # Append the model's function call part.
                contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
                # Append the api response part.
                contents.append(types.Content(role="user", parts=[api_response_part]))
            else:
                # The model gave a natural language response
                function_calling_in_process = False
                break # No more parts in response.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\\$")))

api.refill_rpm  15


# Ask a question

<span style="font-size:18px;">
    If you're on free-tier of Gemini you probably want to Run-before here. Your usage tier can be configured in the api-helper at the top of the notebook.
</span>

In [26]:
send_message("What is the current session for US exchanges?")

#### Natural language response

The current market session for US exchanges is closed.


In [27]:
send_message("What is the US market status?")

#### Natural language response

The US market is currently closed as of Sun Jun 8 01:51:23 2025 America/New_York time. There is no holiday today. The market session is closed.


In [28]:
send_message("When was the last US market close?")

api.on_error.next_model: model is now  gemini-2.0-flash-exp


#### Natural language response

The last market close for the US exchange was on Friday, June 6, 2025, at 8:00 PM local time.


In [30]:
send_message("What is Apple's stock ticker?")

#### Natural language response

The stock ticker for Apple is AAPL.


In [32]:
send_message("What is the current price of Amazon stock?")

#### Natural language response

The current price of Amazon (AMZN) stock is \\$213.57. The price changed by \\$5.66, which is a 2.7223% increase. The high price of the day is \\$213.8699, and the low price of the day is \\$210.5. The opening price of the day was \\$212.4, and the previous close price was \\$207.91. The price was last updated on Sat Jun 7 16:00:00 2025.


In [34]:
send_message("show me Apple's basic financials")

#### Natural language response

Here is a summary of Apple's basic financials:

**Financial Highlights:**

*   **Cash Flow:** The trailing twelve months cash flow per share is 6.86253.
*   **Dividends:** The current trailing twelve months dividend yield is 0.5028%. The indicated annual dividend yield is 0.5183671.
*   **Profitability:**
    *   The trailing twelve months net profit margin is 24.3%.
    *   The trailing twelve months operating margin is 31.81%.
*   **Valuation:**
    *   The price to book value is 45.5972.
    *   The trailing twelve months price to earnings ratio is 31.3042.
*   **Asset Utilization:** The asset turnover for the trailing twelve months is 1.1673.
*   **Debt:** The long term debt to equity ratio annually is 1.5057, and quarterly is 1.1762.
*   **Market Performance:**
    *   The 52 week high was 260.1, recorded on December 26, 2024.
    *   The 52 week low was 169.2101, recorded on April 8, 2025.

**Additional Information:**

*   The enterprise value is 3115737.8 million.
*   The market capitalization is 3045713.8 million.


In [39]:
send_message("I need Apple's daily candlestick from 2025-05-06")

api.zero_error: model is now  gemini-2.0-flash


#### Natural language response

On May 6, 2025, Apple's stock (AAPL) had the following daily candlestick data:
*   After-hours price: 201.15
*   Closing price: 198.51
*   High price: 200.65
*   Low price: 197.02
*   Opening price: 198.21
*   Pre-market price: 198.65
*   Volume: 51,216,482


In [41]:
send_message("Tell me who are Apple's peers?")

#### Natural language response

The peers of Apple, as identified by their sub-industry, are Dell Technologies - C (DELL), Super Micro Computer Inc (SMCI), Hewlett Packard Enterprise (HPE), HP Inc (HPQ), NetApp Inc (NTAP), Western Digital Corp (WDC), Pure Storage Inc - Class A (PSTG), and IonQ Inc (IONQ).


In [45]:
send_message("Show me the recommendation trends for all of Apple's peers")

#### Natural language response

Here is a summary of the recommendation trends for Apple's peers:

*   **Dell Technologies (DELL):** The recommendation trends for Dell Technologies show a consistent outlook from March 2025 to June 2025. The consensus is positive, with a majority of analysts recommending a "buy". Specifically, the number of "buy" recommendations ranges from 20 to 21, "hold" recommendations remain stable at 4, and there are 5 to 6 "strong buy" recommendations. There are no "sell" or "strong sell" recommendations.
*   **Super Micro Computer (SMCI):** The recommendation trends for Super Micro Computer show a generally positive sentiment. "Buy" recommendations increased from 7 in March 2025 to 10 in June 2025. "Hold" recommendations also increased from 8 in March 2025 to 10 in June 2025. There are 2 "sell" recommendations and 2 "strong buy" recommendations across the months.
*   **Hewlett Packard Enterprise (HPE):** The recommendation trends for Hewlett Packard Enterprise show a stable outlook from March 2025 to June 2025. The consensus is positive, with a mix of "buy" and "hold" recommendations. The number of "buy" recommendations is either 6 or 7, "hold" recommendations are either 8 or 9, and there are 4 "strong buy" recommendations. There are no "sell" or "strong sell" recommendations.
*   **HP Inc (HPQ):** The recommendation trends for HP Inc show a consistent outlook from March 2025 to June 2025. The majority of analysts recommend a "hold". Specifically, there are 4 "buy" recommendations, 13 "hold" recommendations, 1 "sell" recommendation, and 2 "strong buy" recommendations across the months.
*   **NetApp (NTAP):** The recommendation trends for NetApp show a stable outlook with a slight increase in "buy" recommendations from March 2025 to June 2025. The number of "buy" recommendations increased from 8 to 9, "hold" recommendations decreased from 17 to 16, and there are 3 "strong buy" recommendations. There are no "sell" or "strong sell" recommendations.
*   **Western Digital (WDC):** The recommendation trends for Western Digital show a positive sentiment with increasing "buy" recommendations from March 2025 to June 2025. The number of "buy" recommendations increased from 15 to 19, "hold" recommendations decreased from 9 to 5, and there are 6 "strong buy" recommendations. There are no "sell" or "strong sell" recommendations.
*   **Pure Storage (PSTG):** The recommendation trends for Pure Storage show a consistent outlook from March 2025 to June 2025. The consensus is positive, with a majority of analysts recommending a "buy". Specifically, there are 13 "buy" recommendations, 6 "hold" recommendations, 1 "sell" recommendation, and 8 "strong buy" recommendations across the months.
*   **IonQ (IONQ):** The recommendation trends for IonQ show a stable outlook from March 2025 to June 2025. The consensus is positive, with a majority of analysts recommending a "buy". Specifically, there are 7 "buy" recommendations, 2 "hold" recommendations, and 2 "strong buy" recommendations across the months. There are no "sell" or "strong sell" recommendations.

In [43]:
send_message("Tell me who are Amazon's peers?")

#### Natural language response

Amazon's peers in the subIndustry grouping are: Coupang Inc (CPNG), Ebay Inc (EBAY), Ollie's Bargain Outlet Holdings (OLLI), Etsy Inc (ETSY), Dillards Inc-Cl A (DDS), Macy's Inc (M), Savers Value Village Inc (SVV), and Groupon Inc (GRPN).


api.refill_rpm  15


In [51]:
#api.set_default_model(3) # generate with gemini-2.5-flash-preview
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Round prices to two decimal places.
    Present the data with multiple columns for display in markdown.
    Discuss and provide details about any patterns you notice in the price data.""")
api.set_default_model(0)

#### Natural language response

The current share price of Amazon (AMZN) is \\$213.57 as of June 7, 2025.

Here is the candlestick data for the past month (May 7, 2025 to June 6, 2025), sorted in descending order by date:

| Date       | Open    | High    | Low     | Close   | Volume      |
| ---------- | ------- | ------- | ------- | ------- | ----------- |
| 2025-06-06 | \\$212.40 | \\$213.87 | \\$210.50 | \\$213.57 | 39,832,500  |
| 2025-06-05 | \\$209.55 | \\$212.81 | \\$207.56 | \\$207.91 | 51,979,243  |
| 2025-06-04 | \\$206.55 | \\$208.18 | \\$205.18 | \\$207.23 | 29,915,592  |
| 2025-06-03 | \\$207.11 | \\$208.95 | \\$205.03 | \\$205.71 | 33,139,121  |
| 2025-06-02 | \\$204.98 | \\$207.00 | \\$202.68 | \\$206.65 | 29,113,319  |
| 2025-05-30 | \\$208.03 | \\$208.81 | \\$204.23 | \\$205.70 | 34,700,005  |
| 2025-05-29 | \\$203.09 | \\$206.69 | \\$202.19 | \\$206.02 | 34,892,044  |
| 2025-05-28 | \\$205.92 | \\$207.66 | \\$204.41 | \\$204.72 | 28,549,753  |
| 2025-05-27 | \\$204.84 | \\$205.99 | \\$201.70 | \\$205.01 | 51,679,406  |
| 2025-05-23 | \\$211.45 | \\$211.93 | \\$208.85 | \\$210.25 | 38,492,128  |
| 2025-05-22 | \\$211.08 | \\$214.84 | \\$210.10 | \\$211.37 | 56,193,682  |
| 2025-05-21 | \\$206.45 | \\$206.88 | \\$202.67 | \\$205.17 | 64,347,317  |
| 2025-05-20 | \\$210.71 | \\$211.66 | \\$205.75 | \\$208.64 | 75,205,042  |
| 2025-05-19 | \\$201.65 | \\$206.62 | \\$201.26 | \\$206.16 | 34,314,810  |
| 2025-05-16 | \\$204.63 | \\$205.59 | \\$202.65 | \\$204.07 | 29,470,373  |
| 2025-05-15 | \\$201.38 | \\$205.76 | \\$200.16 | \\$203.10 | 38,938,882  |
| 2025-05-14 | \\$198.90 | \\$202.37 | \\$197.85 | \\$200.99 | 33,393,545  |
| 2025-05-13 | \\$201.61 | \\$203.46 | \\$200.06 | \\$201.12 | 42,460,924  |
| 2025-05-12 | \\$206.85 | \\$206.85 | \\$204.37 | \\$205.59 | 43,318,478  |
| 2025-05-09 | \\$193.38 | \\$194.69 | \\$191.16 | \\$193.06 | 29,663,143  |
| 2025-05-08 | \\$191.43 | \\$194.33 | \\$188.82 | \\$192.08 | 41,043,620  |
| 2025-05-07 | \\$185.56 | \\$190.99 | \\$185.01 | \\$188.71 | 44,002,926  |

**Summary:**

Over the past month, Amazon's stock price has shown an overall upward trend, starting from \\$188.71 on May 7, 2025, and closing at \\$213.57 on June 6, 2025. There have been fluctuations throughout the month, with periods of both increases and decreases in price. Notably, there was a significant jump in price between May 7 and May 22, followed by a period of consolidation and further gains towards the end of the month. The trading volume has also varied, with higher volumes often coinciding with larger price movements.


In [50]:
send_message("What is Apple's ticker overview")

#### Natural language response

Apple Incorporated, ticker symbol AAPL, is a major player in the stock market, listed on the XNAS exchange. The company's Central Index Key (CIK) is 0000320193, its composite Financial Instrument Global Identifier (FIGI) is BBG000B9XRY4, and its share class FIGI is BBG001S5N8V8. With a market capitalization of 3,012,556,104,200 United States dollars, Apple has a broad portfolio of hardware and software products. The company employs 164,000 people. Apple's initial listing date was December 12, 1980.


In [53]:
send_message("Tell me about Amazon's historical and current recommendation trends")

#### Natural language response

As of June 8, 2025, the recommendation trends for Amazon (AMZN) are as follows:

In June 2025, there are 50 buy recommendations, 24 strong buy recommendations and 5 hold recommendations.
In May 2025, there are 51 buy recommendations, 22 strong buy recommendations and 6 hold recommendations.
In April 2025, there are 50 buy recommendations, 23 strong buy recommendations and 4 hold recommendations.
In March 2025, there are 51 buy recommendations, 21 strong buy recommendations and 5 hold recommendations.
There are no sell or strong sell recommendations in any of the months listed.


api.zero_error: model is now  gemini-2.0-flash


In [55]:
send_message("What is Google's stock ticker symbol?")

#### Natural language response

Google's stock ticker symbols are GOOGL and GOOG on the NASDAQ stock exchange. It is also listed on the Frankfurt Stock Exchange under the ticker symbol GGQ1. These symbols refer to Alphabet Inc., Google's holding company. Google's initial public offering (IPO) took place on August 19, 2004, opening on the NASDAQ National Market under the ticker symbol GOOGL.


In [56]:
#api.set_default_model(3) # generate with gemini-2.5-flash-preview
send_message("What is MGM Studio's stock symbol?")
api.set_default_model(0)

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


api.on_error.next_model: model is now  gemini-2.0-flash-exp


#### Natural language response

I was unable to find the stock symbol for Mgm Studios.

In [57]:
#api.set_default_model(3) # generate with gemini-2.5-flash-preview
send_message("What is Amazon MGM Studio's stock symbol?")
api.set_default_model(0)

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


#### Natural language response

Amazon MGM Studios is a subsidiary of Amazon, and Amazon's stock symbol is AMZN.


api.zero_error: model is now  gemini-2.0-flash


In [26]:
send_message("What is Facebook's stock ticker symbol?")

Score similarity to stored grounding: 0it [00:00, ?it/s]


#### Natural language response

Facebook's stock ticker symbol is META and it is listed on the NASDAQ.


api.refill_rpm  15


In [61]:
send_message(
    '''Tell me about Amazon's current bullish versus bearish predictions, and recommendation trends.
    Include a discussion of any short-term trends, and sentiment analysis.''')

#### Natural language response

As of June 8, 2025, here's a summary of Amazon's bullish versus bearish predictions and recommendation trends:

**Recommendation Trends:**

*   Based on analyst recommendations, the trend is strongly bullish. In June 2025, there are 24 strong buy recommendations, 51 buy recommendations, and 5 hold recommendations. There are no sell or strong sell recommendations.
*   The number of strong buy recommendations has increased slightly from 22 in May 2025 to 24 in June 2025, indicating a strengthening bullish sentiment.

**Sentiment Analysis:**

*   The news articles from May 8, 2025, to June 1, 2025, show a predominantly positive sentiment toward Amazon.
*   Many articles highlight Amazon's strong position in e-commerce, cloud computing (Amazon Web Services), and digital advertising as key drivers of growth.
*   Several articles suggest that Amazon is a good buy for long-term investors, citing its attractive valuation and growth potential in AI and other emerging technologies.
*   Some articles mention potential headwinds, such as tariffs and increased competition, but overall, the sentiment remains positive due to Amazon's diversified business model and strong market position.
*   Bill Ackman's Pershing Square has taken a new stake in Amazon, adding to the bullish sentiment around the stock.
*   Several articles highlight Amazon's investments in AI and its potential to become a \\$5 trillion company.

**Short-Term Trends:**

*   The news articles suggest that Amazon's stock has recovered nearly 30% since its April low, driven by improving trade relations between the U.S. and China.
*   Analysts are bullish on the stock, citing margin expansion, leadership in cloud and advertising, and cost discipline.
*   However, some articles mention near-term uncertainties, such as slowing growth in online sales and potential regulatory crackdowns.

**Overall Summary:**

The overall sentiment towards Amazon is strongly bullish, with analysts recommending a buy rating and numerous articles highlighting the company's growth potential in various sectors. While there are some short-term uncertainties, the long-term outlook for Amazon remains positive due to its diversified business model, strong market position, and investments in emerging technologies like AI.


In [63]:
send_message(
    '''Tell me about Google's share price over the past month.
    Perform a sentiment analysis of news during the same period. Include trends.''')

Score similarity to stored grounding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of Google's (Alphabet Inc.) stock price and news sentiment over the past month (May 8, 2025, to June 6, 2025):

**Ticker Symbols:** Alphabet Inc. has two ticker symbols: GOOGL (Class A shares with voting rights) and GOOG (Class C shares with no voting rights).

**News Sentiment:**
The news sentiment surrounding Alphabet has been largely positive, focusing on the company's advancements in artificial intelligence (AI) and its potential impact on future growth. Here's a breakdown of the key themes:

*   **AI Advancements:** Many articles highlight Alphabet's progress in AI, particularly showcased at the Google I/O event. This includes AI-powered search, Android XR, and the Gemini AI model. These advancements are seen as a way for Google to fend off competition and drive future growth.
*   **Undervalued Stock:** Several articles suggest that Alphabet's stock is undervalued, trading at a discount compared to its AI rivals and the S\&P 500 average. This is attributed to investors underestimating Alphabet's AI capabilities and ecosystem.
*   **Analyst Confidence:** Analysts and institutional investors express strong confidence in Alphabet's long-term prospects, citing its robust business and growth drivers in AI, cloud computing, and YouTube.
*   **Potential Challenges:** Some articles mention potential challenges, such as Apple's exploration of AI-powered search alternatives and potential antitrust regulation. However, these concerns are often balanced by the company's strong financial position and diversified business model.
*   **Waymo as a Growth Driver:** Alphabet's autonomous taxi service, Waymo, is highlighted as a potential hidden driver for the company's future growth and partnerships.
*   **Saudi Investment:** Alphabet is among the companies involved in joint tech investments with Saudi Arabia.
*   **Bill Ackman's Investment:** Bill Ackman's hedge fund has a significant portion of its portfolio invested in Alphabet, indicating his belief in the stock.
*   **Cathie Wood's Prediction:** Cathie Wood, CEO of ARK Invest, believes that AI will disrupt traditional search engines like Google.

**Mixed Sentiment:**
Some articles present mixed sentiment, acknowledging both the opportunities and challenges facing Alphabet. For example, one article discusses the potential impact of Apple introducing AI-powered search tools in Safari on Alphabet's search dominance but argues that Alphabet's strong integration across various services makes it unlikely for Apple's move to significantly disrupt Google's search business.

**Overall:**
The news sentiment suggests that Alphabet is well-positioned to benefit from the growth in AI, with its strong financial position, diversified business model, and investments in next-generation technologies. While there are some potential challenges, analysts and investors remain largely confident in the company's long-term prospects.

**Disclaimer:** I am an AI chatbot and cannot provide financial advice. This information is for informational purposes only.


In [64]:
send_message(
    '''How is the outlook for Apple based on trends and news sentiment over the past month?
    Perform the same analysis on all of Apple's peers. Then compare Apple result to it's peers.''')

Upsert chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]


limited 5/min, waiting 46.17237114906311s
api.refill_rpm  15


Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

The outlook for Apple (AAPL) is mixed based on recent trends and news sentiment.

**Recommendation Trends:**
*   Analyst recommendations for Apple show a generally positive sentiment, with a majority of analysts rating the stock as a "buy" or "strong buy." In June 2025, there were 25 "buy" ratings, 13 "hold" ratings, 15 "strong buy" ratings, 3 "sell" ratings, and 1 "strong sell" rating.

**News Sentiment:**
*   The news sentiment surrounding Apple is mixed. Some articles highlight positive aspects such as Warren Buffett's continued investment in Apple, the potential for growth in the services segment, and the company's strong brand and ecosystem.
*   However, there are also concerns about potential tariffs, slowing iPhone sales growth, and increasing competition in the AI space. Some articles suggest that Apple's valuation is high and that the stock may be overvalued.

**Peer Analysis:**
Here's a summary of the news sentiment for Apple's peers over the past month:

*   **Dell Technologies (DELL):** The sentiment is generally positive, with mentions of its involvement in energy efficiency standards, the growing storage systems market, and its AI server business.
*   **Super Micro Computer (SMCI):** The sentiment is mixed. While the company is benefiting from the AI hardware industry and has a partnership with the Saudi government, it has also faced accounting issues and experienced stock declines due to disappointing results.
*   **Hewlett Packard Enterprise (HPE):** The sentiment is generally positive, with mentions of its involvement in energy efficiency standards, the growing storage systems market, and its collaboration with Nvidia on AI solutions.
*   **HP Inc (HPQ):** The sentiment is negative, with the company reporting mixed earnings and cutting its outlook.
*   **NetApp Inc (NTAP):** The sentiment is neutral, with mentions of its presence in the growing storage systems market but no specific details about its performance.
*   **Western Digital Corp (WDC):** The sentiment is neutral, with mentions of its presence in the growing storage systems market but no specific details about its performance.
*   **Pure Storage Inc (PSTG):** The sentiment is generally positive, with expectations of margin expansion and increased demand due to AI.
*   **IonQ Inc (IONQ):** The sentiment is negative, with concerns about its high valuation, unimpressive sales, and the uncertainty around the practical applications of quantum computing.

**Comparative Summary:**

*   Apple's recommendation trends are generally positive, but the news sentiment is mixed due to concerns about tariffs and competition.
*   Among Apple's peers, Dell Technologies and Hewlett Packard Enterprise have generally positive news sentiment, while HP Inc has negative sentiment. Super Micro Computer and IonQ have mixed to negative sentiment. NetApp and Western Digital have neutral sentiment.
*   Overall, Apple's outlook is comparable to its peers in the hardware and IT infrastructure space, with both positive and negative factors influencing investor sentiment.

In conclusion, Apple's outlook is cautiously optimistic, with potential for growth in the services segment and AI, but also facing challenges from tariffs and competition. The peer analysis reveals a mixed sentiment in the industry, with some companies showing strong growth potential while others face challenges.


In [66]:
send_message(
    '''What does the recent news say about Apple and the impact of tariffs? From 2025-03-01 up to today.
    Discuss and provide details about any patterns you notice in the price data. Ignore duplicate entry.''')

#### Natural language response

The recent news regarding Apple (AAPL) and the impact of tariffs between March 1, 2025, and June 6, 2025, reveals a complex situation with both challenges and attempts to mitigate negative effects. Here's a summary:

**Key Themes:**

*   **Tariff Impact on Earnings:** Several articles mention that tariffs are expected to cost Apple hundreds of millions to nearly \\$1 billion in upcoming quarters, impacting net income growth.
*   **Production Shifts:** Apple is reportedly shifting some production to countries like India and Vietnam to avoid tariffs on Chinese imports.
*   **Potential Cost Increases:** Tariffs could lead to a significant surge in iPhone production costs, potentially forcing Apple to raise prices or move production to the U.S., which analysts deem unfeasible.
*   **Trade Policy Uncertainty:** President Trump's trade policies and tariff threats create uncertainty for Apple, potentially increasing costs and forcing price increases.
*   **Temporary Relief:** Apple has received temporary exemptions from some tariffs, providing some relief, but a semiconductor-specific tariff is expected in the near future.
*   **Analyst Concerns:** Analysts have expressed concerns about the impact of tariffs on Apple's profitability, sales growth, and valuation, leading to price target cuts.
*   **Market Volatility:** Announcements regarding tariffs and trade tensions have caused market sell-offs, impacting Apple's stock price.
*   **Positive Developments:** There have been some positive developments, such as a temporary pause on elevated tariff rates and potential court rulings against tariffs, which have led to rallies in Apple's stock.
*   **AI and Innovation:** Apple is reportedly developing AI-powered smart glasses and working on Apple Intelligence to increase iPhone sales, indicating a focus on innovation to drive growth.

**Patterns in Price Data (based on news sentiment):**

*   **Negative Sentiment:** News of increased tariffs, potential cost increases, and negative analyst outlooks generally correlate with a sinking stock price for Apple.
*   **Positive Sentiment:** Temporary tariff exemptions, potential court rulings against tariffs, and positive developments in trade talks tend to lead to a surge in Apple's stock price.
*   **Uncertainty:** Overall, the news suggests that Apple's stock price is highly sensitive to developments in trade policy and tariff announcements, leading to volatility.

**Summary:**

Apple faces significant challenges due to tariffs and trade tensions, which are expected to impact its earnings and operations. The company is taking steps to mitigate these effects by shifting production and focusing on innovation. However, uncertainty remains, and Apple's stock price is likely to be volatile in response to developments in trade policy.


api.refill_rpm  15


# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>

# Update June 7, 2025

<span style="font-size:18px;">
    Bugfix version 102 finally brings Essy to a stable milestone. A month and a half late :) There's still more to be built including adding reasoning, agents, and structured output. A few unimplemented rest endpoints remain that could make Essy more self-reliant. The vector store has gotten bigger but not smarter. Essy can tell us pre-scored news has some sentiment but cannot generate it due to limited summaries. Essy can detect interesting patterns in a dataset but not between adjacent datasets. There's so much data we'll need to recruit Essy some help.
</span>

# Advanced (localhost required)

<span style="font-size:18px;">
    The functions demonstrated here require a locally running notebook. Additional dependencies include <a href="https://www.selenium.dev/documentation/grid/getting_started/">selenium-server</a> running on localhost. A dedicated GPU with at least 8GB VRAM is recommended but not required. Output is generated with Gemma 3 12B QAT, Gemma.cpp, and (later) Gemma 3n. Output on Kaggle is based on cached data.
</span>

In [ ]:
# soon